In [2]:
import trimesh
import numpy as np
import pandas as pd
import plotly.graph_objects as go

## 1.Map the distribution of different types of neurons across the brain

In [ ]:
csv_file = 'dataframe_data/meta_data_mannual_check.csv'

data = pd.read_csv(csv_file)

Unipolar = data[data['mannual_check'] == 'Unipolar']
Bipolar = data[data['mannual_check'] == 'Bipolar']
Multipolar = data[data['mannual_check'] == 'Multipolar']
# soma_out = data #[data['in_brain'] == 'No']
# soma_in = data[data['in_brain'] == 'Yes']

coordinates_unipolar = Unipolar[['projected_x', 'projected_y', 'projected_z']].values
coordinates_bipolar = Bipolar[['projected_x', 'projected_y', 'projected_z']].values
coordinates_multipolar = Multipolar[['projected_x', 'projected_y', 'projected_z']].values
# coordinates_soma_out = soma_out[['projected_x', 'projected_y', 'projected_z']].values
# coordinates_soma_in = soma_in[['soma_x', 'soma_y', 'soma_z']].values

mesh = trimesh.load_mesh(r'H:\code\BrainParcellation-main\data\fly_mesh\whole_mesh\merge_all.obj')

vertices = mesh.vertices
faces = mesh.faces

fig = go.Figure()

fig.add_trace(go.Mesh3d(
    x=vertices[:, 0],
    y=vertices[:, 1],
    z=vertices[:, 2],
    i=faces[:, 0],
    j=faces[:, 1],
    k=faces[:, 2],
    opacity=0.5,
    color='white',
    name="Mesh"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_unipolar[:, 0],  
    y=coordinates_unipolar[:, 1],  
    z=coordinates_unipolar[:, 2],  
    mode='markers',
    marker=dict(size=1, color='red', opacity=1),
    name="Unipolar"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_bipolar[:, 0],  
    y=coordinates_bipolar[:, 1],  
    z=coordinates_bipolar[:, 2],  
    mode='markers',
    marker=dict(size=2, color='green', opacity=1),
    name="Bipolar"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_multipolar[:, 0],  
    y=coordinates_multipolar[:, 1],  
    z=coordinates_multipolar[:, 2],  
    mode='markers',
    marker=dict(size=2, color='blue', opacity=1),
    name="Multipolar"
))

# fig.add_trace(go.Scatter3d(
#     x=coordinates_soma_out[:, 0],  
#     y=coordinates_soma_out[:, 1],  
#     z=coordinates_soma_out[:, 2],  
#     mode='markers',
#     marker=dict(size=1, color='blue', opacity=0.2),
#     name="Soma_out"
# ))

# fig.add_trace(go.Scatter3d(
#     x=coordinates_soma_in[:, 0],  
#     y=coordinates_soma_in[:, 1],  
#     z=coordinates_soma_in[:, 2],  
#     mode='markers',
#     marker=dict(size=1, color='red', opacity=0.2),
#     name="Soma_in"
# ))


fig.update_layout(
    title="3D Mesh and Point Clouds by Type",
    scene=dict(
        xaxis=dict(visible=False, showgrid=False),
        yaxis=dict(visible=False, showgrid=False),
        zaxis=dict(visible=False, showgrid=False),
    ),
    autosize=False,  
    width=1000,  
    height=800,  
)

fig.show()

In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull
from tqdm import tqdm  

def load_obj(file_path):
    vertices = []
    faces = []
    
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('v '):  
                vertices.append(list(map(float, line.strip().split()[1:])))
            elif line.startswith('f '): 
                face_data = line.strip().split()[1:]
                face = []
                for data in face_data:
                    indices = data.split('/')
                    face.append(int(indices[0]))  
                faces.append(face)
    
    return np.array(vertices), faces

def create_convex_hull(vertices, soma_coords):
    vertices = vertices[~np.isnan(vertices).any(axis=1)]  
    soma_coords = soma_coords[~np.isnan(soma_coords).any(axis=1)]  
    
    all_points = np.vstack((vertices, soma_coords))
    
    hull = ConvexHull(all_points)
    
    new_vertices = all_points[hull.vertices]
    new_faces = hull.simplices
    
    return new_vertices, new_faces

def create_obj_with_convex_hull(filename, vertices, faces, soma_coords):
    new_vertices, new_faces = create_convex_hull(vertices, soma_coords)
    
    all_vertices = np.vstack((vertices, new_vertices))
    all_vertices = np.unique(all_vertices, axis=0)
    
    updated_faces = []
    for face in new_faces:
        updated_face = []
        for i in face:
            vertex_index = np.where(np.all(all_vertices == new_vertices[i], axis=1))[0][0]
            updated_face.append(vertex_index + 1)  
        updated_faces.append(updated_face)
    
    with open(filename, 'w') as obj_file:
        for vertex in tqdm(vertices, desc="Writing original vertices", unit="vertex"):
            obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
        
        for face in tqdm(faces, desc="Writing original faces", unit="face"):
            obj_file.write(f"f {' '.join(map(str, face))}\n")
        
        for vertex in tqdm(new_vertices, desc="Writing convex hull vertices", unit="vertex"):
            obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
        
        for face in tqdm(updated_faces, desc="Writing convex hull faces", unit="face"):
            obj_file.write(f"f {' '.join(map(str, face))}\n")
        

mesh_file = r'G:\energency_fly\data\fly_mesh\whole_mesh\merge_all.obj'
vertices, faces = load_obj(mesh_file)

csv_file = r'G:\energency_fly\data\synapse_coordinates\meta_data.csv'
data = pd.read_csv(csv_file)
soma_out = data[data['in_brain'] == 'No']
soma_coords = soma_out[['soma_x', 'soma_y', 'soma_z']].values

create_obj_with_convex_hull("soma_and_mesh_hull.obj", vertices, faces, soma_coords)

## 2.Map the distribution of different types of cells in specific brain regions

In [ ]:
csv_file = r'G:\energency_fly\data\synapse_coordinates\meta_data.csv'
data = pd.read_csv(csv_file)

selected_region = 'LA_L'

Unipolar = data[(data['type'] == 'Unipolar') & (data['Soma_region'] == f'{selected_region}')]
Bipolar = data[(data['type'] == 'Bipolar') & (data['Soma_region'] == f'{selected_region}')]
Multipolar = data[(data['type'] == 'Multipolar') & (data['Soma_region'] == f'{selected_region}')]

coordinates_unipolar = Unipolar[['projected_x', 'projected_y', 'projected_z']].values
coordinates_bipolar = Bipolar[['projected_x', 'projected_y', 'projected_z']].values
coordinates_multipolar = Multipolar[['projected_x', 'projected_y', 'projected_z']].values

mesh = trimesh.load_mesh(f'G:\\energency_fly\\data\\fly_mesh\\data\\{selected_region}.obj')

vertices = mesh.vertices
faces = mesh.faces

fig = go.Figure()

fig.add_trace(go.Mesh3d(
    x=vertices[:, 0],
    y=vertices[:, 1],
    z=vertices[:, 2],
    i=faces[:, 0],
    j=faces[:, 1],
    k=faces[:, 2],
    opacity=0.5,
    color='white',
    name="Mesh"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_unipolar[:, 0],  
    y=coordinates_unipolar[:, 1],  
    z=coordinates_unipolar[:, 2],  
    mode='markers',
    marker=dict(size=1, color='red', opacity=1),
    name="Unipolar"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_bipolar[:, 0],  
    y=coordinates_bipolar[:, 1],  
    z=coordinates_bipolar[:, 2],  
    mode='markers',
    marker=dict(size=1, color='green', opacity=1),
    name="Bipolar"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_multipolar[:, 0],  
    y=coordinates_multipolar[:, 1],  
    z=coordinates_multipolar[:, 2],  
    mode='markers',
    marker=dict(size=1, color='blue', opacity=1),
    name="Multipolar"
))

fig.update_layout(
    title="3D Mesh and Point Clouds by Type",
    scene=dict(
        xaxis=dict(visible=False, showgrid=False),
        yaxis=dict(visible=False, showgrid=False),
        zaxis=dict(visible=False, showgrid=False),
    ),
    autosize=False,  
    width=1000,  
    height=800,  
)

fig.show()

In [ ]:
import pandas as pd
import trimesh
import plotly.graph_objects as go

left_brain = ['AMMC_L','FLA_L','CAN_L','AL_L','LH_L','MB_CA_L','MB_PED_L','MB_VL_L','MB_ML_L','BU_L','GA_L','LAL_L','SLP_L','SIP_L','SMP_L','CRE_L',
             'SCL_L','ICL_L','IB_L','ATL_L','VES_L','EPA_L','GOR_L','SPS_L','IPS_L','AOTU_L','AVLP_L','PVLP_L','PLP_L','WED_L','ME_L','AME_L','LO_L',
             'LOP_L','LA_L']
right_brain = ['AMMC_R','FLA_R','CAN_R','AL_R','LH_R','MB_CA_R','MB_PED_R','MB_VL_R','MB_ML_R','BU_R','GA_R','LAL_R','SLP_R','SIP_R','SMP_R','CRE_R',
               'SCL_R','ICL_R','IB_R','ATL_R','VES_R','EPA_R','GOR_R','SPS_R','IPS_R','AOTU_R','AVLP_R','PVLP_R','PLP_R','WED_R','ME_R','AME_R','LO_R',
               'LOP_R','LA_R',]
middle_brain = ['FB','EB','PB','NO','PRW','SAD','GNG','OCG']

selected_regions = ['FB','EB','PB','NO','PRW','SAD','GNG','OCG']

# Load CSV data
csv_file = r'G:\energency_fly\data\synapse_coordinates\meta_data.csv'
data = pd.read_csv(csv_file)

# Create the figure
fig = go.Figure()

# Loop through each selected region and plot the mesh and points
for selected_region in selected_regions:
    # Filter data by region and cell type
    Unipolar = data[(data['type'] == 'Unipolar') & (data['Soma_region'] == selected_region)]
    Bipolar = data[(data['type'] == 'Bipolar') & (data['Soma_region'] == selected_region)]
    Multipolar = data[(data['type'] == 'Multipolar') & (data['Soma_region'] == selected_region)]

    # Extract coordinates
    coordinates_unipolar = Unipolar[['projected_x', 'projected_y', 'projected_z']].values
    coordinates_bipolar = Bipolar[['projected_x', 'projected_y', 'projected_z']].values
    coordinates_multipolar = Multipolar[['projected_x', 'projected_y', 'projected_z']].values

    # Load the mesh for the selected region
    mesh = trimesh.load_mesh(f'G:\\energency_fly\\data\\fly_mesh\\data\\{selected_region}.obj')

    vertices = mesh.vertices
    faces = mesh.faces

    # Add mesh to the figure
    fig.add_trace(go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        opacity=0.5,
        color='white',
        name=f"Mesh_{selected_region}"
    ))

    # Add Unipolar points to the figure
    fig.add_trace(go.Scatter3d(
        x=coordinates_unipolar[:, 0],  
        y=coordinates_unipolar[:, 1],  
        z=coordinates_unipolar[:, 2],  
        mode='markers',
        marker=dict(size=1, color='red', opacity=1),
        name=f"Unipolar_{selected_region}"
    ))

    # Add Bipolar points to the figure
    fig.add_trace(go.Scatter3d(
        x=coordinates_bipolar[:, 0],  
        y=coordinates_bipolar[:, 1],  
        z=coordinates_bipolar[:, 2],  
        mode='markers',
        marker=dict(size=1, color='green', opacity=1),
        name=f"Bipolar_{selected_region}"
    ))

    # Add Multipolar points to the figure
    fig.add_trace(go.Scatter3d(
        x=coordinates_multipolar[:, 0],  
        y=coordinates_multipolar[:, 1],  
        z=coordinates_multipolar[:, 2],  
        mode='markers',
        marker=dict(size=1, color='blue', opacity=1),
        name=f"Multipolar_{selected_region}"
    ))

# Update the layout
fig.update_layout(
    title="3D Mesh and Point Clouds by Type for Multiple Regions",
    scene=dict(
        xaxis=dict(visible=False, showgrid=False),
        yaxis=dict(visible=False, showgrid=False),
        zaxis=dict(visible=False, showgrid=False),
    ),
    autosize=False,  
    width=1000,  
    height=800,  
)

fig.show()

## 3. Create a voxel file containing subregions

In [ ]:
import numpy as np
import SimpleITK as sitk
import plotly.graph_objects as go
import plotly.express as px

def read_nrrd_file(file_path):
    image = sitk.ReadImage(file_path)
    volume = sitk.GetArrayFromImage(image)  
    return volume

def plot_3d_scatter_plotly(volume):
    x, y, z = np.indices(volume.shape)
    unique_values = np.unique(volume)

    scatter_data = []

    colors = px.colors.qualitative.Set1

    color_map = {value: colors[i % len(colors)] for i, value in enumerate(unique_values) if value != 0}

    for value, color in color_map.items():
        if value == 0:  
            continue
        
        coords = np.column_stack(np.where(volume == value))
        
        scatter_data.append(
            go.Scatter3d(
                x=coords[:, 0], 
                y=coords[:, 1], 
                z=coords[:, 2], 
                mode='markers',
                marker=dict(
                    size=2, 
                    color=color,  
                    opacity=0.6
                ),
                name=str(value)  
            )
        )

    fig = go.Figure(data=scatter_data)

    fig.update_layout(
        title="3D Scatter Plot of Voxel Values",
        scene=dict(
        xaxis=dict(visible=False, showgrid=False),
        yaxis=dict(visible=False, showgrid=False),
        zaxis=dict(visible=False, showgrid=False),
    ),
    autosize=False,  
    width=1000,  
    height=800,
    )

    fig.show()

nrrd_file_path = r'G:\energency_fly\code\BrainParcellation-main\BrainParcellation-main\microenviron\output_full\parc_region25.nrrd'  

volume = read_nrrd_file(nrrd_file_path)
plot_3d_scatter_plotly(volume)

## 4. Mapping neurons to the brain

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import trimesh

def read_swc(file_path):
    swc_data = pd.read_csv(file_path, delim_whitespace=True, comment='#', header=None)
    swc_data = swc_data.replace(['NA', np.nan], -1)
    swc_data = swc_data.to_numpy()
    swc_data = np.core.records.fromarrays(swc_data.T,
                                          names='id, type, x, y, z, radius, parent',
                                          formats='i4, i4, f4, f4, f4, f4, i4')
    swc_data['x'] /= 1000
    swc_data['y'] /= 1000
    swc_data['z'] /= 1000
    swc_data['radius'] /= 1000

    return swc_data

def create_mesh_from_swc(swc_data, minRadius=0.005, sphere_subdivisions=0, cylinder_sections=3):
    meshes = []

    for i, node in enumerate(swc_data):
        sphere = trimesh.creation.icosphere(subdivisions=sphere_subdivisions, radius=max(node['radius'], minRadius))
        sphere.apply_translation([node['x'], node['y'], node['z']])
        meshes.append(sphere)

        if node['parent'] != -1:
            parent_node = swc_data[np.where(swc_data['id'] == node['parent'])[0][0]]

            start = np.array([node['x'], node['y'], node['z']])
            end = np.array([parent_node['x'], parent_node['y'], parent_node['z']])
            length = np.linalg.norm(end - start)

            if length > 0:
                direction = (end - start) / length
                radius = (max(node['radius'], minRadius) + max(parent_node['radius'], minRadius)) / 2

                cylinder = trimesh.creation.cylinder(radius=max(radius, minRadius), height=length, sections=cylinder_sections)

                try:
                    cylinder.apply_transform(trimesh.geometry.align_vectors([0, 0, 1], direction))
                except np.linalg.LinAlgError:
                    axis = np.cross([0, 0, 1], direction)
                    angle = np.arccos(np.dot([0, 0, 1], direction))
                    cylinder.apply_transform(trimesh.transformations.rotation_matrix(angle, axis))

                cylinder.apply_translation((start + end) / 2)

                meshes.append(cylinder)

    combined_mesh = trimesh.util.concatenate(meshes)

    return combined_mesh

def save_mesh_to_obj(mesh, output_path):
    mesh.export(output_path)

def batch_convert_swc_to_obj(input_dir, output_dir, sphere_subdivisions=0, cylinder_sections=3):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    swc_files = glob.glob(os.path.join(input_dir, "*.swc"))

    for swc_file in swc_files:
        swc_data = read_swc(swc_file)
        mesh = create_mesh_from_swc(swc_data, sphere_subdivisions=sphere_subdivisions, cylinder_sections=cylinder_sections)
        obj_filename = os.path.basename(swc_file).replace('.swc', '.obj')
        obj_path = os.path.join(output_dir, obj_filename)
        save_mesh_to_obj(mesh, obj_path)
        print(f"Converted {swc_file} to {obj_path}")

input_dir = r'G:\energency_fly\data\test'
output_dir = r'G:\energency_fly\data\test\converted_obj'

batch_convert_swc_to_obj(input_dir, output_dir, sphere_subdivisions=0, cylinder_sections=3)

In [ ]:
import os
import glob
import plotly.graph_objects as go
import trimesh

def load_and_plot_mesh(obj_path, fig, mesh_color='blue', opacity=0.5):
    mesh = trimesh.load_mesh(obj_path)
    vertices = mesh.vertices
    faces = mesh.faces
    mesh_trace = go.Mesh3d(
        x=vertices[:, 0],
        y=vertices[:, 1],
        z=vertices[:, 2],
        i=faces[:, 0],
        j=faces[:, 1],
        k=faces[:, 2],
        opacity=opacity,
        color=mesh_color,
        name=os.path.splitext(os.path.basename(obj_path))[0],
        showlegend=True
    )
    fig.add_trace(mesh_trace)

def plot_multiple_meshes(obj_file_paths):
    fig = go.Figure()
    
    colors = ['white', 'red', 'green', 'yellow', 'purple', 'orange', 'cyan', 'magenta']  # Color list
    for i, obj_path in enumerate(obj_file_paths):
        color = colors[i % len(colors)]  
        opacity = 0.3 if i == 0 else 1
        load_and_plot_mesh(obj_path, fig, mesh_color=color, opacity=opacity)

    # data = np.array([[382.164, 266.108, 188.960], 
    #             [387.776, 262.328, 186.320], 
    #             [401.472, 292.960, 167.480]])
    # data = np.array([[399.072, 286.880, 169.680]
    #             ])
    # data = np.array([[389.852, 279.424, 177.960], 
    #             [406.368, 291.104, 169.960] 
    #             ])
    data = np.array([[322.508, 249.720, 213.080], 
                [391.360, 271.072, 178.960] 
                ])
    
    fig.add_trace(go.Scatter3d(
    x=data[:, 0],  
    y=data[:, 1],  
    z=data[:, 2],  
    mode='markers',
    marker=dict(size=7, color='red', opacity=1),
    name="Soma_in"
     ))
    
    fig.update_layout(
        title="3D Mesh Display",
        scene=dict(
            xaxis=dict(visible=False, showgrid=False),
            yaxis=dict(visible=False, showgrid=False),
            zaxis=dict(visible=False, showgrid=False),
        ),
        autosize=False,
        width=1000,
        height=800,
    )
    fig.show()

obj_files = glob.glob(r'G:\energency_fly\data\test\converted_obj\*.obj')
merge_all_obj_path = r'G:\energency_fly\data\fly_mesh\whole_mesh\merge_all.obj'
obj_files.insert(0, merge_all_obj_path)
plot_multiple_meshes(obj_files)

## 5. Different brain regions were mapped to divide the whole brain

In [ ]:
selected_regions1 = ['LO_L','LA_L','ME_L','AME_L','WED_L']
selected_regions2 = ['MB_VL_L','IPS_L','LAL_L','SPS_L','VES_L']
selected_regions3 = ['AL_L','CRE_L','AMMC_L','GNG']

mesh1 = trimesh.load_mesh(f'G:\\energency_fly\\data\\fly_mesh\\merge_all.obj')


vertices1 = mesh1.vertices
faces1 = mesh1.faces

fig = go.Figure()

fig.add_trace(go.Mesh3d(
    x=vertices1[:, 0],
    y=vertices1[:, 1],
    z=vertices1[:, 2],
    i=faces1[:, 0],
    j=faces1[:, 1],
    k=faces1[:, 2],
    opacity=0.5,
    color='white',
    name="Mesh1"
))

for selected_region in selected_regions1:
    mesh2 = trimesh.load_mesh(f'G:\\energency_fly\\data\\fly_mesh\\data\\{selected_region}.obj')
    vertices2 = mesh2.vertices
    faces2 = mesh2.faces
    fig.add_trace(go.Mesh3d(
        x=vertices2[:, 0],
        y=vertices2[:, 1],
        z=vertices2[:, 2],
        i=faces2[:, 0],
        j=faces2[:, 1],
        k=faces2[:, 2],
        opacity=0.7,
        color='blue',
        name="Mesh2"
    ))

for selected_region in selected_regions2:
    mesh3 = trimesh.load_mesh(f'G:\\energency_fly\\data\\fly_mesh\\data\\{selected_region}.obj')
    vertices3 = mesh3.vertices
    faces3 = mesh3.faces
    fig.add_trace(go.Mesh3d(
        x=vertices3[:, 0],
        y=vertices3[:, 1],
        z=vertices3[:, 2],
        i=faces3[:, 0],
        j=faces3[:, 1],
        k=faces3[:, 2],
        opacity=0.7,
        color='green',
        name="Mesh3"
    ))

for selected_region in selected_regions3:
    mesh4 = trimesh.load_mesh(f'G:\\energency_fly\\data\\fly_mesh\\data\\{selected_region}.obj')
    vertices4 = mesh4.vertices
    faces4 = mesh4.faces
    fig.add_trace(go.Mesh3d(
        x=vertices4[:, 0],
        y=vertices4[:, 1],
        z=vertices4[:, 2],
        i=faces4[:, 0],
        j=faces4[:, 1],
        k=faces4[:, 2],
        opacity=0.7,
        color='red',
        name="Mesh4"
    ))

fig.update_layout(
    title="3D Mesh and Point Clouds by Type",
    scene=dict(
        xaxis=dict(visible=False, showgrid=False),
        yaxis=dict(visible=False, showgrid=False),
        zaxis=dict(visible=False, showgrid=False),
    ),
    autosize=False,  
    width=1000,  
    height=800,  
)

fig.show()

## 6.The distribution of neurons Module1/2/3 was drawn

In [10]:
csv_file1 = 'dataframe_data/heatmap_data.csv'

data = pd.read_csv(csv_file1)
Module1_index = data.iloc[:106, 0]
Module2_index = data.iloc[106:, 0]
Module2a_index = data.iloc[106:225, 0]
Module2b_index = data.iloc[225:, 0]

Module2b1_index = data.iloc[225:254, 0]
Module2b2_index = data.iloc[254:299, 0]
Module2b3_index = data.iloc[299:339, 0]
Module2b4_index = data.iloc[339:443, 0]
Module2b5_index = data.iloc[443:, 0]

csv_file2 = 'dataframe_data/meta_test.csv'
meta_data = pd.read_csv(csv_file2, dtype={0:str})
Module1 = meta_data[meta_data['region_id_me'].isin(Module1_index)]
Module2 = meta_data[meta_data['region_id_me'].isin(Module2_index)]
Module2a = meta_data[meta_data['region_id_me'].isin(Module2a_index)]
Module2b = meta_data[meta_data['region_id_me'].isin(Module2b_index)]

Module2b1 = meta_data[meta_data['region_id_me'].isin(Module2b1_index)]
Module2b2 = meta_data[meta_data['region_id_me'].isin(Module2b2_index)]
Module2b3 = meta_data[meta_data['region_id_me'].isin(Module2b3_index)]
Module2b4 = meta_data[meta_data['region_id_me'].isin(Module2b4_index)]
Module2b5 = meta_data[meta_data['region_id_me'].isin(Module2b5_index)]

In [ ]:
coordinates_Module1 = Module1[['x', 'y', 'z']].values / 1000
coordinates_Module2 = Module2[['x', 'y', 'z']].values / 1000
coordinates_Module2a = Module2a[['x', 'y', 'z']].values / 1000
coordinates_Module2b = Module2b[['x', 'y', 'z']].values / 1000
coordinates_Module2b1 = Module2b1[['x', 'y', 'z']].values / 1000
coordinates_Module2b2 = Module2b2[['x', 'y', 'z']].values / 1000
coordinates_Module2b3 = Module2b3[['x', 'y', 'z']].values / 1000
coordinates_Module2b4 = Module2b4[['x', 'y', 'z']].values / 1000
coordinates_Module2b5 = Module2b5[['x', 'y', 'z']].values / 1000

mesh = trimesh.load_mesh(r'G:\code\BrainParcellation-main\data\fly_mesh\whole_mesh\merge_all.obj')

vertices = mesh.vertices
faces = mesh.faces

fig = go.Figure()

fig.add_trace(go.Mesh3d(
    x=vertices[:, 0],
    y=vertices[:, 1],
    z=vertices[:, 2],
    i=faces[:, 0],
    j=faces[:, 1],
    k=faces[:, 2],
    opacity=0.2,
    color='white',
    name="Mesh"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_Module1[:, 0],  
    y=coordinates_Module1[:, 1],  
    z=coordinates_Module1[:, 2],  
    mode='markers',
    marker=dict(size=1.5, color='orange', opacity=1),
    name="Module1"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_Module2[:, 0],  
    y=coordinates_Module2[:, 1],  
    z=coordinates_Module2[:, 2],  
    mode='markers',
    marker=dict(size=1.5, color='green', opacity=1),
    name="Module2"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_Module2a[:, 0],  
    y=coordinates_Module2a[:, 1],  
    z=coordinates_Module2a[:, 2],  
    mode='markers',
    marker=dict(size=1, color='red', opacity=1),
    name="Module2a"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_Module2b[:, 0],  
    y=coordinates_Module2b[:, 1],  
    z=coordinates_Module2b[:, 2],  
    mode='markers',
    marker=dict(size=1, color='#97D7F2', opacity=1),
    name="Module2b"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_Module2b1[:, 0],  
    y=coordinates_Module2b1[:, 1],  
    z=coordinates_Module2b1[:, 2],  
    mode='markers',
    marker=dict(size=1, color='#07AEE3', opacity=1),
    name="Module2b1"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_Module2b2[:, 0],  
    y=coordinates_Module2b2[:, 1],  
    z=coordinates_Module2b2[:, 2],  
    mode='markers',
    marker=dict(size=1, color='#B3D49D', opacity=1),
    name="Module2b2"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_Module2b3[:, 0],  
    y=coordinates_Module2b3[:, 1],  
    z=coordinates_Module2b3[:, 2],  
    mode='markers',
    marker=dict(size=1, color='#DC4959', opacity=1),
    name="Module2b3"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_Module2b4[:, 0],  
    y=coordinates_Module2b4[:, 1],  
    z=coordinates_Module2b4[:, 2],  
    mode='markers',
    marker=dict(size=1, color='#EAA5C2', opacity=1),
    name="Module2b4"
))

fig.add_trace(go.Scatter3d(
    x=coordinates_Module2b5[:, 0],  
    y=coordinates_Module2b5[:, 1],  
    z=coordinates_Module2b5[:, 2],  
    mode='markers',
    marker=dict(size=1, color='#626FB3', opacity=1),
    name="Module2b5"
))


fig.update_layout(
    title="3D Mesh and Point Clouds by Type",
    scene=dict(
        xaxis=dict(visible=False, showgrid=False),
        yaxis=dict(visible=False, showgrid=False),
        zaxis=dict(visible=False, showgrid=False),
    ),
    autosize=False,  
    width=1000,  
    height=1000,  
)

fig.show()